# [문제]
- law_2.docx 파일을 읽고, Chroma 저장
- LLM 질문 -> 답변
- 전세사기피해에 관한 법률 질문만 받기
- 이 외의 질문은 '답변을 할 수 없습니다.' 

In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader

In [2]:
## 1. 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)
document_list

[Document(metadata={'source': 'law_2.docx'}, page_content='전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\nA×(B/C)\n\n- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  \n\n- **B**: 전세사기피해주택의 가격  \n\n- **C**: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액\n\n\n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신

In [3]:
len(document_list)

6

In [4]:
## 2. 임베딩 -> 벡터 데이터베이스에 저장
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

## 2.3. 벡터 데이터베이스에 저장
## [방법 1] in memory
database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
)

database

In [5]:
## 3. 질문 -> 벡터 데이터베이스(vector store)에서 유사도 검색
## 3.1. 사용자 질문
# query = '전세사기피해자는 누구인가요?'
query = '오늘 점심 메뉴 정해줘~'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)
retrieved_docs

[Document(id='4eac851a-39ec-4563-aeef-caf593dac571', metadata={'source': 'law_2.docx'}, page_content='전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\nA×(B/C)\n\n- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  \n\n- **B**: 전세사기피해주택의 가격  \n\n- **C**: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합

In [6]:
len(retrieved_docs)

2

In [7]:
retrieved_docs[0].page_content

'전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\nA×(B/C)\n\n- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  \n\n- **B**: 전세사기피해주택의 가격  \n\n- **C**: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액\n\n\n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액\n\n 

In [8]:
a = '!'
a += '가'
a

'!가'

In [9]:
## 문서 객체에서 문자열만 추출
## [방법 1]
ex = ''

for doc in retrieved_docs:
    # print(doc.page_content + '😊😊\n')
    ex += doc.page_content + '😊😊\n\n'

ex

'전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\nA×(B/C)\n\n- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  \n\n- **B**: 전세사기피해주택의 가격  \n\n- **C**: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액\n\n\n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액\n\n 

In [10]:
[doc.page_content for doc in retrieved_docs]

['전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\nA×(B/C)\n\n- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  \n\n- **B**: 전세사기피해주택의 가격  \n\n- **C**: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액\n\n\n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액\n\n

In [3]:
var = (num for num in ['a', 'b'])
var

<generator object <genexpr> at 0x0000018015279770>

In [4]:
type(var)

generator

In [5]:
tu = 'a', 'b'
type(tu)

tuple

In [ ]:
var = (num for num in ['a', 'b'])
var

<generator object <genexpr> at 0x0000018015279770>

In [ ]:
'😀'.join(num for num in ['a', 'b'])

'a😀b'

In [2]:
'😀'.join([num for num in ['a', 'b']])

'a😀b'

In [11]:
## 리스트내포
lt = [i for i in range(11)]
lt

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [12]:
## 일반 for문
lt2 = []

for i in range(11):
    lt2.append(i)

lt2

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [13]:
a

'!가'

In [14]:
## 문자열: join 함수
'~'.join(['하나', '둘', '셋'])

'하나~둘~셋'

In [15]:
## [방법 2]
## 리스트 내 for문
context = '😊😊\n\n'.join([doc.page_content for doc in retrieved_docs])
context

'전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\nA×(B/C)\n\n- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  \n\n- **B**: 전세사기피해주택의 가격  \n\n- **C**: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액\n\n\n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액\n\n 

In [16]:
## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달
## 4.1. 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.
- 전세사기피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formmated_prompt = prompt.format(
    # retrieved_docs=retrieved_docs,
    retrieved_docs=context,
    query=query,
)

formmated_prompt

"\n[identity]\n- 당신은 전세사기피해 법률 전문가입니다.\n- [context]를 참고하여 사용자의 질문에 답변해주세요.\n- 전세사기피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.\n\n[context]\n전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\nA×(B/C)\n\n- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  \n\n- **B**: 전세사기피해주택의 가격  \n\n- **C**

In [17]:
## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서 보내기
ai_message = llm.invoke(formmated_prompt)
ai_message

AIMessage(content='답변을 할 수 없습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 1911, 'total_tokens': 1918, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BexDsPRaUJykCGxDM2agOvXa48iDx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--99d32527-4a19-4c5c-b708-a439b4453681-0', usage_metadata={'input_tokens': 1911, 'output_tokens': 7, 'total_tokens': 1918, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})